In [1]:
import pandas as pd
import numpy as np

In [2]:
-0.6548
-0.6317
-0.3686
-0.1090
5
0.0539
6
293.6831
62
0.2412

0.2412

In [3]:
water = pd.read_csv('water.txt', sep = '\t')
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [4]:
mortality = water.mortality.values
hardness = water.hardness.values

In [5]:
sum_1 = 0
sum_2 = 0
sum_3 = 0

for mort, hard in zip(mortality, hardness):
    sum_1 += (mort - np.mean(mortality)) * (hard - np.mean(hardness))
    sum_2 += (mort - np.mean(mortality)) ** 2
    sum_3 += (hard - np.mean(hardness)) ** 2
    
print sum_1 / (sum_2 * sum_3) ** 0.5

-0.6548486232042464


In [6]:
from scipy.stats import pearsonr
pearsonr(mortality, hardness)

(-0.6548486232042465, 1.0331339260349571e-08)

In [7]:
from scipy.stats import spearmanr
spearmanr(mortality, hardness)

SpearmanrResult(correlation=-0.6316646189166502, pvalue=4.79546153722838e-08)

In [8]:
south = water[water.location == 'South']
south_pear = pearsonr(south.mortality.values, south.hardness.values)

north = water[water.location == 'North']
north_pear = pearsonr(north.mortality.values, north.hardness.values)

print south_pear
print north_pear

(-0.6021532715484158, 0.0011346743048161927)
(-0.3685978383288718, 0.029344659047110606)


In [9]:
a = 203
b = 718
c = 239
d = 515
print (a * d - b * c) / ((a + b) * (a + c) * (b + d) * (d + c)) ** 0.5
from sklearn.metrics import matthews_corrcoef 
print matthews_corrcoef([a, c], [b, d])

-0.109002374587
0.0


In [10]:
from scipy.stats import chi2_contingency
chi2_contingency([[a, b], [c, d]])

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[243.03402985, 677.96597015],
        [198.96597015, 555.03402985]]))

In [11]:
chi2_contingency([[48, 30], [92, 36]])

(1.9271792226870357, 0.16506738276246485, 1, array([[53.00970874, 24.99029126],
        [86.99029126, 41.00970874]]))

In [12]:
chi2_contingency([[197, 111, 33], [382, 685, 331], [110, 342, 333]])

(293.68311039689746,
 2.4964299580093467e-62,
 4,
 array([[ 93.08597464, 153.74722662,  94.16679873],
        [381.6251981 , 630.318542  , 386.0562599 ],
        [214.28882726, 353.93423138, 216.77694136]]))

In [13]:
import scipy.stats as ss
def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

In [14]:
cramers_corrected_stat(np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]]))

0.2412013934500338

In [15]:
import scipy

fem = np.append(np.ones(203), np.zeros(718))
mal = np.append(np.ones(239), np.zeros(515))

In [16]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [17]:
print proportions_diff_confint_ind(mal, fem)

(0.053905233215813156, 0.13922183141523897)


In [18]:
proportions_diff_z_test(proportions_diff_z_stat_ind(mal, fem))

8.153453089576601e-06

# Test 2

Выберите задачи, в которых обязательно нужно применять поправку на множественную проверку гипотез

* Сравнение эффективности лечения пациентов в подгруппах по большому количеству признаков

* Выбор инвестиционных фондов с помощью сравнения доходности каждого из них с доходностью базового актива по нескольким историческим периодам

* Попарное сравнение средних большого количества выборок

* Локализация различий в активности мозга испытуемых в разных экспериментальных условиях

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:
        Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [22]:
aucs = pd.read_csv('AUCs.txt', sep = '\t')
aucs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [59]:
from scipy.stats import wilcoxon
from itertools import combinations
result = 10000000
answer = ''

counter = 0
p = []
for comb in combinations(['C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf'], 2):
    cur_result = wilcoxon(- aucs[comb[0]] + aucs[comb[1]])
    print cur_result, comb
    if cur_result.statistic < result:
        result = cur_result.statistic
        answer = comb
    if cur_result.pvalue < 0.05:
        counter += 1
    p.append(cur_result.pvalue)
print answer

WilcoxonResult(statistic=6.5, pvalue=0.01075713311978963) ('C4.5', 'C4.5+m')
WilcoxonResult(statistic=43.0, pvalue=0.861262330095348) ('C4.5', 'C4.5+cf')
WilcoxonResult(statistic=11.0, pvalue=0.015906444101703374) ('C4.5', 'C4.5+m+cf')
WilcoxonResult(statistic=17.0, pvalue=0.046332729793395394) ('C4.5+m', 'C4.5+cf')
WilcoxonResult(statistic=22.0, pvalue=0.3278256758446406) ('C4.5+m', 'C4.5+m+cf')
WilcoxonResult(statistic=10.0, pvalue=0.022909099354356588) ('C4.5+cf', 'C4.5+m+cf')
('C4.5', 'C4.5+m')


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [60]:
counter

4

Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [57]:
from statsmodels.sandbox.stats.multicomp import multipletests 
reject, p_corrected, a1, a2 = multipletests(p, 
                                            alpha = 0.05, 
                                            method = 'holm') 
print reject

[False False False False False False]


Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [58]:
from statsmodels.sandbox.stats.multicomp import multipletests 
reject, p_corrected, a1, a2 = multipletests(p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
print reject

[ True False  True False False  True]


Насколько корректно, на ваш взгляд, применение метода Бенджамини-Хохберга в этой задаче?

Всегда корректно, это же метод Бенджамини-Хохберга

* Некорректно: статистики, соответствующие разным гипотезам, зависимы, так что предположения метода Бенджамини-Хохберга не выполняются

Корректно: статистики, соответствующие разным гипотезам, независимы, так что предположения метода Бенджамини-Хохберга выполняются

Мы подозреваем, что в проведённом с C4.5 эксперименте на самом деле классификаторы сильнее отличаются друг от друга, просто нам не удалось это заметить. Что можно сделать, чтобы увеличить вероятность обнаружения различий, если они действительно существуют?

* Взять больше датасетов

Закрыть глаза на эффект множественной проверки гипотез и не делать никакой поправки

Попробовать настроить больше гиперпараметро

# Test 3

О каждой из них мы знаем:

сколько детей она родила (признак ceb)
возраст (age)
длительность получения образования (educ)
религиозная принадлежность (religion)
идеальное, по её мнению, количество детей в семье (idlnchld)
была ли она когда-нибудь замужем (evermarr)
возраст первого замужества (agefm)
длительность получения образования мужем (heduc)
знает ли она о методах контрацепции (knowmeth)
использует ли она методы контрацепции (usemeth)
живёт ли она в городе (urban)
есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)
Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?


In [115]:
raw = pd.read_csv("botswana.tsv", sep="\t", index_col=False) 
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [116]:
print raw.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64


Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [117]:
len(raw.dropna())

1834

In [118]:
raw.drop('evermarr', axis=1, inplace=True)

In [119]:
raw['nevermarr'] = raw['agefm'].apply(lambda x : 0 if x > 0 else 1)

In [120]:
raw['agefm'].fillna(0, inplace=True)

In [123]:
raw.loc[raw.nevermarr == 1,'heduc'] = -1

In [124]:
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [135]:
len(raw) - len(raw['heduc'].dropna())

0

In [136]:
raw['heduc_noans'] = raw['heduc'].apply(lambda x : 0 if x > -2 else 1)

In [137]:
raw['heduc'].fillna(-2, inplace=True)

In [138]:
raw['idlnchld_noans'] = raw['idlnchld'].apply(lambda x : 0 if x > -1 else 1)

In [139]:
raw['idlnchld'].fillna(-1, inplace=True)

In [140]:
raw['usemeth_noans'] = raw['usemeth'].apply(lambda x : 0 if x > -1 else 1)
raw['usemeth'].fillna(-1, inplace=True)

In [142]:
raw.dropna(inplace=True)

In [155]:
raw.shape[0] * raw.shape[1]

78264

Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации 
R
2
 ? Округлите до трёх знаков после десятичной точки.

In [156]:
import statsmodels.formula.api as smf

In [157]:
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,heduc_noans,idlnchld_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [ ]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth +'\
                    'agefm + heduc + urban + electric + radio + tv + bicycle +'\
                    'nevermarr + heduc_noans + idlnchld_noans + usemeth_noans', 
             data=raw)
fitted = m1.fit()
print fitted.summary()

Обратите внимание, что для признака religion в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?

In [160]:
3

3

Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

In [163]:
import statsmodels.stats.api as sms
print 'Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [164]:
raw.drop('religion', axis=1, inplace=True)
raw.drop('radio', axis=1, inplace=True)
raw.drop('tv', axis=1, inplace=True)

In [165]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth +'\
                    'agefm + heduc + urban + electric + bicycle +'\
                    'nevermarr + heduc_noans + idlnchld_noans + usemeth_noans', 
             data=raw)
fitted = m2.fit()
print "F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit())

F=0.919236, p=0.467231, k1=5.000000


Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением 
c
u
s
e
m
e
t
h
=
−
1
c 
usemeth
​	 =−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 
5
.
5
×
1
0
−
8
5.5×10 
−8
 , нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [166]:
raw.drop('usemeth_noans', axis=1, inplace=True)
raw.drop('usemeth', axis=1, inplace=True)

In [168]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth +'\
                    'agefm + heduc + urban + electric + bicycle +'\
                    'nevermarr + heduc_noans + idlnchld_noans', 
             data=raw)
fitted = m3.fit()
print m2.fit().compare_f_test(m3.fit())

(92.890582301098021, 3.1552009480371243e-40, 2.0)
